In [1]:
import logging
import time

# Configure logger
logger: logging.Logger = logging.getLogger("register_model_logger")
logger.setLevel(logging.INFO)
logger.propagate = False  # Prevent duplicate logs from parent loggers

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [2]:
start_time = time.time()  

logger.info("Notebook execution started.")

2025-07-09 01:47:03 - INFO - Notebook execution started.


## Model Service for Code Generation 

In this section, we demonstrate how to deploy a code generation service that provides a REST API endpoint for generating code based on natural language queries. This service provides a REST API endpoint for generating code based on natural language queries, 
The service includes performance optimization features to handle large repositories:
- **Parameter-based operation control** with `metadata_only` mode for fast processing 
- **Resource optimization** with batched processing and configurable timeouts

### API Usage Examples

The optimized code generation service can be invoked through its REST API using the following patterns:

#### Details

The API includes some features like:

1. **metadata_only mode**: Allows extracting only basic metadata without running the resource-intensive LLM analysis
2. **Timeout controls**: Prevents worker timeouts with configurable processing limits

```bash
# Example 1: Direct code generation (no repository)
curl -X 'POST' \
  'https://endpoint/invocations' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "inputs": {
    "question": "Write Python code to load the LLM model using Ollama with \'llama3\' and generate an inspirational quote."
  }
}'

# Example 2: Repository-enhanced code generation with metadata-only mode (FAST)
curl -X 'POST' \
  'https://endpoint/invocations' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "inputs": {
    "question": "Write Python code to scrape a website and extract all links",
    "repository_url": "https://github.com/MunGell/awesome-for-beginners"
  }
}'

# Example 3: Repository-enhanced code generation with full processing (DEEP UNDERSTANDING)
curl -X 'POST' \
  'https://endpoint/invocations' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "inputs": {
    "question": "Write Python code to scrape a website and extract all links",
    "repository_url": "https://github.com/MunGell/awesome-for-beginners"
  }
}'
```

The service will automatically adapt to the parameters provided:
- With `metadata_only=true`, it performs lightweight processing ideal for large repositories
- With full processing, it performs deep analysis but take longer

In [3]:
%%time

# === Imports ===
import os
import sys
import logging
import mlflow

# Configure logging
logging.basicConfig(
    level=logging.INFO, 
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)
logger = logging.getLogger(__name__)

# === Extend system path to include parent directory for module resolution ===
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

# Import utility functions and CodeGenerationService
from src.utils import configure_hf_cache
from core.code_generation_service import CodeGenerationService

# === Configuration settings ===
CONFIG_PATH = "../configs/config.yaml"
SECRETS_PATH = "../configs/secrets.yaml"
MLFLOW_MODEL_NAME = "Code-Generation-Model"
LOCAL_MODEL_PATH = "/home/jovyan/datafabric/meta-llama3.1-8b-Q8/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"

# Configure HuggingFace cache for the embedding model
configure_hf_cache()

# Define embedding model parameters
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"
EMBEDDING_MODEL_PATH = os.path.join(
    os.environ.get("HF_HOME", "/home/jovyan/local/hugging_face"),
    "embedding_models", 
    EMBEDDING_MODEL_NAME
)

mlflow.set_tracking_uri('/phoenix/mlflow')
# Set up the MLflow experiment
mlflow.set_experiment(MLFLOW_MODEL_NAME)

# Check if the model file exists
if not os.path.exists(LOCAL_MODEL_PATH):
    logger.info(f"⚠️ Warning: Model file not found at {LOCAL_MODEL_PATH}. Please verify the path.")

# Verify the locally saved embedding model exists
if not os.path.exists(EMBEDDING_MODEL_PATH):
    # If the embedding model wasn't saved earlier, do it now
    try:
        from sentence_transformers import SentenceTransformer
        os.makedirs(os.path.dirname(EMBEDDING_MODEL_PATH), exist_ok=True)
        logger.info(f"Downloading and saving embedding model to {EMBEDDING_MODEL_PATH}...")
        model = SentenceTransformer(EMBEDDING_MODEL_NAME)
        model.save(EMBEDDING_MODEL_PATH)
        logger.info(f"Embedding model saved successfully.")
    except Exception as e:
        logger.error(f"Error saving embedding model: {str(e)}")
        logger.warning("Will proceed without a local embedding model. The service will download it during initialization.")
        EMBEDDING_MODEL_PATH = None

# Use the CodeGenerationService's log_model method to register the model in MLflow
with mlflow.start_run(run_name=MLFLOW_MODEL_NAME) as run:
    # Log and register the model using the service's classmethod
    CodeGenerationService.log_model(
        secrets_path=SECRETS_PATH,
        config_path=CONFIG_PATH,
        model_path=LOCAL_MODEL_PATH,
        embedding_model_path=EMBEDDING_MODEL_PATH,
        delay_async_init=True  # Since this service requires parallelism we use this parameter to avoid error when picling the model
    )
    
    # Register the model in MLflow Model Registry
    model_uri = f"runs:/{run.info.run_id}/code_generation_service"
    mlflow.register_model(
        model_uri=model_uri,
        name=MLFLOW_MODEL_NAME
    )
    
    logger.info(f"✅ Model registered successfully with run ID: {run.info.run_id}")

/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/model.py:172: UserWarning: Type hint used in the model's predict function is not supported for MLflow's schema validation. Type hints must be wrapped in list[...] because MLflow assumes the predict method to take multiple input instances. Specify your type hint as `list[typing.Dict[str, typing.Any]]` for a valid signature. Remove the type hint to disable this warning. To enable validation for the input data, specify input example or model signature when logging the model. 
  func_info = _get_func_info_if_type_hint_supported(predict_attr)
2025/07/09 01:47:13 INFO mlflow.tracking.fluent: Experime

2025/07/09 01:50:45 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.21.2, required: mlflow==2.9.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025-07-09 01:53:34 - INFO - Model and artifacts successfully registered in MLflow.
Successfully registered model 'Code-Generation-Model'.
Created version '1' of model 'Code-Generation-Model'.
2025-07-09 01:53:35 - INFO - ✅ Model registered successfully with run ID: 91b457ec0fac4e3187f3e4b49359cb45


CPU times: user 19.5 s, sys: 39.1 s, total: 58.6 s
Wall time: 6min 32s


In [4]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")

2025-07-09 01:53:35 - INFO - ⏱️ Total execution time: 6m 32.32s
2025-07-09 01:53:35 - INFO - ✅ Notebook execution completed successfully.


Built with ❤️ using [**HP AI Studio**](https://hp.com/ai-studio).